## Creating Dummy Dataset

In [1]:
import pandas as pd
import random
import numpy as np

# preferences and activities for users and guides
PREFERENCES = ['historical', 'adventure', 'beaches', 'nature', 'museums', 'art', 'hiking', 'food', 'shopping']

# Generating 200 users with random preferences and budgets
def generate_users(n):
    users = []
    for i in range(1, n + 1):
        name = f"User_{i}"
        preferences = random.sample(PREFERENCES, random.randint(2, 4)) 
        budget = random.randint(500, 3000) 
        users.append({'user_id': i, 'name': name, 'preferences': preferences, 'budget': budget, 'itinerary': []})
    return pd.DataFrame(users)

# Generating 50 guides with random knowledge and prices
def generate_guides(n):
    guides = []
    for i in range(1, n + 1):
        name = f"Guide_{i}"
        knowledge = random.sample(PREFERENCES, random.randint(2, 3)) 
        rating = round(random.uniform(3.5, 5.0), 1) 
        price_per_day = random.randint(150, 1000)  
        guides.append({'guide_id': i, 'name': name, 'knowledge': knowledge, 'rating': rating, 'price_per_day': price_per_day})
    return pd.DataFrame(guides)

# Creating data for 200 users and 50 guides
users_data = generate_users(200)
guides_data = generate_guides(50)


In [2]:
users_data.head()

,user_id,name,preferences,budget,itinerary
0,1,User_1,"[shopping, food, nature, art]",1800,[]
1,2,User_2,"[nature, food]",1702,[]
2,3,User_3,"[food, adventure, museums]",1679,[]
3,4,User_4,"[historical, art, food, adventure]",779,[]
4,5,User_5,"[beaches, museums, food, historical]",2850,[]


In [3]:
guides_data.head()

,guide_id,name,knowledge,rating,price_per_day
0,1,Guide_1,"[historical, museums, shopping]",4.8,950
1,2,Guide_2,"[hiking, art, adventure]",4.7,550
2,3,Guide_3,"[shopping, food, beaches]",4.2,482
3,4,Guide_4,"[food, art]",3.9,928
4,5,Guide_5,"[hiking, beaches]",3.9,405


##  User Registration and Profile Setup

In [6]:
def register_user(name, preferences, budget):
    global users_data 
    new_user_id = users_data['user_id'].max() + 1
    new_user = pd.DataFrame([{'user_id': new_user_id, 'name': name, 'preferences': preferences, 'budget': budget, 
                              'itinerary': []}])
    users_data = pd.concat([users_data, new_user], ignore_index=True)
    print(f"User {name} registered successfully!")

# Register a new user
register_user('Vivek', ['historical', 'adventure'], 1200)


User Vivek registered successfully!


## Itinerary Planning

In [7]:
def create_itinerary(user_id, destinations, activities):
    # Create the itinerary as a list of destinations and activities
    itinerary = {'destinations': destinations, 'activities': activities}
    
    # Find the user by user_id and update their itinerary with a dictionary
    users_data.loc[users_data['user_id'] == user_id, 'itinerary'] = [itinerary]
    
    print(f"Itinerary for user {user_id} created successfully!")

# Example 1: sam plans a trip
create_itinerary(1, ['Paris', 'Rome'], ['museum', 'hiking'])


# Example 2: john plans a trip
create_itinerary(2, ['Jaipur', 'Goa'], ['historical', 'beaches'])




Itinerary for user 1 created successfully!
Itinerary for user 2 created successfully!


In [8]:
import json
# Function to display user itineraries
def display_itinerary(user_id):
    user_itinerary = users_data[users_data['user_id'] == user_id]
    if not user_itinerary.empty:
        print(f"\nUser {user_itinerary.iloc[0]['name']}'s itinerary:")
        print(json.dumps(user_itinerary.iloc[0]['itinerary'], indent=4))
    else:
        print(f"\nNo itinerary found for user {user_id}")

# Display the data for User 1
display_itinerary(1)

# Display the data for User 2
display_itinerary(2)


User User_1's itinerary:
{
    "destinations": [
        "Paris",
        "Rome"
    ],
    "activities": [
        "museum",
        "hiking"
    ]
}

User User_2's itinerary:
{
    "destinations": [
        "Jaipur",
        "Goa"
    ],
    "activities": [
        "historical",
        "beaches"
    ]
}


## Local Guide Matching Algorithm

In [9]:
def match_guides(user_id):
    user = users_data[users_data['user_id'] == user_id].iloc[0]
    user_preferences = set(user['preferences'])
    budget = user['budget']
    
    print(f"User: {user['name']} (User ID: {user_id})")
    print(f"Preferences: {user['preferences']}")
    print(f"Budget: {budget}")
    print("\nTop Matching Guides:\n")
    
   
    matching_guides = []
    
    for _, guide in guides_data.iterrows():
        guide_knowledge = set(guide['knowledge'])
        
        # Match based on knowledge overlap and budget constraints
        if user_preferences.intersection(guide_knowledge) and guide['price_per_day'] <= budget:
            # Calculate score: preference match + rating/price ratio (custom ranking)
            knowledge_overlap = len(user_preferences.intersection(guide_knowledge)) 
            price_rating_ratio = guide['rating'] / guide['price_per_day'] 
            
            score = knowledge_overlap + price_rating_ratio  # Custom score based on overlap and affordability
            
            matching_guides.append({
                'guide_id': guide['guide_id'],
                'name': guide['name'],
                'knowledge': guide['knowledge'],
                'rating': guide['rating'],
                'price_per_day': guide['price_per_day'],
                'score': score
            })
    
  
    if matching_guides:
        # Sort guides first by score (descending), then by rating (descending), and finally by price (ascending)
        sorted_guides = sorted(matching_guides, key=lambda x: (-x['score'], -x['rating'], x['price_per_day']))
        top_guides = sorted_guides[:5]  # Limit to top 5 matches
        
        for guide in top_guides:
            print(f"Guide: {guide['name']} (Guide ID: {guide['guide_id']})")
            print(f"Knowledge: {guide['knowledge']}")
            print(f"Rating: {guide['rating']}, Price per day: {guide['price_per_day']}, Score: {guide['score']:.2f}")
            print("-" * 40)
    else:
        print(f"No matching guides found for {user['name']}")

# Example: Match guides for a user with ID 5
match_guides(5)


User: User_5 (User ID: 5)
Preferences: ['beaches', 'museums', 'food', 'historical']
Budget: 2850

Top Matching Guides:

Guide: Guide_14 (Guide ID: 14)
Knowledge: ['beaches', 'shopping', 'food']
Rating: 4.6, Price per day: 201, Score: 2.02
----------------------------------------
Guide: Guide_31 (Guide ID: 31)
Knowledge: ['historical', 'museums']
Rating: 3.8, Price per day: 260, Score: 2.01
----------------------------------------
Guide: Guide_10 (Guide ID: 10)
Knowledge: ['beaches', 'food', 'hiking']
Rating: 4.4, Price per day: 363, Score: 2.01
----------------------------------------
Guide: Guide_34 (Guide ID: 34)
Knowledge: ['adventure', 'food', 'historical']
Rating: 3.6, Price per day: 309, Score: 2.01
----------------------------------------
Guide: Guide_28 (Guide ID: 28)
Knowledge: ['historical', 'nature', 'beaches']
Rating: 3.9, Price per day: 362, Score: 2.01
----------------------------------------


## Feedback and Reviews

In [10]:
def leave_review(guide_id, rating):
    global guides_data
    
    # Check if the guide exists
    guide = guides_data[guides_data['guide_id'] == guide_id]
    
    if not guide.empty:
        # Proceed if the guide exists
        guide = guide.iloc[0]  # Get the first (and only) matching guide
        new_rating = round((guide['rating'] + rating) / 2,2)
        guides_data.loc[guides_data['guide_id'] == guide_id, 'rating'] = new_rating
        print(f"New rating for guide {guide['name']}: {new_rating}")
    else:
        print(f"No guide found with guide_id {guide_id}")

# Example: 
leave_review(8, 4.7)


New rating for guide Guide_8: 4.55
